In [1]:
!pip install pyspark

In [2]:
!git clone https://github.com/wandersondsm/teste_engenheiro

Cloning into 'teste_engenheiro'...
remote: Enumerating objects: 1189, done.
remote: Total 1189 (delta 0), reused 0 (delta 0), pack-reused 1189 (from 2)
Receiving objects: 100% (1189/1189), 49.85 MiB | 8.45 MiB/s, done.
Resolving deltas: 100% (1148/1148), done.
Updating files: 100% (1182/1182), done.


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F


In [4]:
spark = SparkSession.builder.appName('Create Medication Request').getOrCreate()

In [5]:
spark

### Funções auxiliares


In [6]:
@F.udf('string')
def remove_uuid(uuid):
  if uuid:
    return uuid.replace('urn:uuid:','')
  return None

### Leitura dos dados e visualização geral

In [7]:
df = spark.read.option('multiline','true').json('/content/teste_engenheiro/data/*.json')

In [8]:
df.show(5)

+--------------------+------------+-----------+
|               entry|resourceType|       type|
+--------------------+------------+-----------+
|[{urn:uuid:071812...|      Bundle|transaction|
|[{urn:uuid:df121e...|      Bundle|transaction|
|[{urn:uuid:f156b8...|      Bundle|transaction|
|[{urn:uuid:b0f49c...|      Bundle|transaction|
|[{urn:uuid:37ff59...|      Bundle|transaction|
+--------------------+------------+-----------+
only showing top 5 rows



In [9]:
df.count()

1180

In [11]:
df_medication = df.select(F.explode('entry'))

In [12]:
df_medication.count()

527113

In [13]:
df_medication.show(5,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
df_save = df_medication.withColumn('RESOURCE_TYPE', F.col('col.resource.resourceType'))\
                       .where("RESOURCE_TYPE like 'MedicationRequest'")\
                       .withColumn('ID', F.col('col.resource.id'))\
                       .withColumn('STATUS', F.col('col.resource.status'))\
                       .withColumn('INTENT', F.col('col.resource.intent'))\
                       .withColumn('AUTHORED_ON', F.col('col.resource.authoredOn').cast('timestamp'))\
                       .withColumn('MEDICATION_CODEABLE_CONCEPT', F.explode(F.col('col.resource.medicationCodeableConcept.coding')))\
                       .withColumn('MEDICATION_CODE',F.col('MEDICATION_CODEABLE_CONCEPT.code'))\
                       .withColumn('MEDICATION_TEXT',F.col('MEDICATION_CODEABLE_CONCEPT.display'))\
                       .withColumn('PATIENT_ID', remove_uuid(F.col('col.resource.subject.reference')))\
                       .withColumn('REQUESTER_REFERERENCE_ID', remove_uuid(F.col('col.resource.requester.reference')))\
                       .withColumn('REQUESTER_NAME', F.col('col.resource.requester.display'))\
                       .drop('col','RESOURCE_TYPE','MEDICATION_CODEABLE_CONCEPT')

In [15]:
df_save.printSchema()

root
 |-- ID: string (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- INTENT: string (nullable = true)
 |-- AUTHORED_ON: timestamp (nullable = true)
 |-- MEDICATION_CODE: string (nullable = true)
 |-- MEDICATION_TEXT: string (nullable = true)
 |-- PATIENT_ID: string (nullable = true)
 |-- REQUESTER_REFERERENCE_ID: string (nullable = true)
 |-- REQUESTER_NAME: string (nullable = true)



In [16]:
df_save.show(5,False)

+------------------------------------+-------+------+-------------------+---------------+----------------------------------------------------------------------------------------------------+------------------------------------+------------------------------------+-------------------------+
|ID                                  |STATUS |INTENT|AUTHORED_ON        |MEDICATION_CODE|MEDICATION_TEXT                                                                                     |PATIENT_ID                          |REQUESTER_REFERERENCE_ID            |REQUESTER_NAME           |
+------------------------------------+-------+------+-------------------+---------------+----------------------------------------------------------------------------------------------------+------------------------------------+------------------------------------+-------------------------+
|eb7f5891-2265-4950-bfc6-7c337be8335d|active |order |1935-09-30 14:04:10|316049         |Hydrochlorothiazide 25 MG             

In [17]:
df_save.coalesce(1).write.option('header',True).option('delimiter',';').csv('medication_request')